In [68]:
from typing import Annotated
from langgraph.graph import StateGraph, START
from langgraph.graph.message import add_messages
from dotenv import load_dotenv
from IPython.display import Image, display
import gradio as gr
from langgraph.prebuilt import ToolNode, tools_condition
import requests
import os
from langchain_openai import ChatOpenAI
from typing import TypedDict




In [69]:
load_dotenv(override=True)

True

In [70]:
from langchain_community.utilities import GoogleSerperAPIWrapper

In [71]:
serper = GoogleSerperAPIWrapper()
serper.run("What is Capital of the US")

"Washington, DC, officially the District of Columbia and commonly known as simply Washington or DC, is the capital city and federal district Washington, D.C., capital of the United States, coextensive with the District of Columbia, located on the northern shore of the Potomac River. Washington, D.C. has been the federal capital of the United States since 1800. Each U.S. state has its own capital city, as do many of its insular areas. Most ... Capital City: Washington, D.C.. Explore the nation's capital as viewed through the collections from the U.S. Capitol to museums and monuments. Capital of the United States of America Washington, DC, the U.S. capital, is a compact city on the Potomac River, bordering the states of ... On July 16, 1790, Washington, DC was established as the nation's capital by the Constitution of the United States. You can read the actual line at the National ... The United States Capitol in Washington, D.C., is a symbol of the American people and our government and

In [72]:
from langchain.agents import Tool

tool_search = Tool(

    name="search",
    func= serper.run,
    description="Userful for searching online"
)

In [73]:
tool_search.invoke("What is the capital of US")

"Washington, DC, officially the District of Columbia and commonly known as simply Washington or DC, is the capital city and federal district Washington, D.C., capital of the United States, coextensive with the District of Columbia, located on the northern shore of the Potomac River. For the capital city, see Washington, D.C.. The United States Capitol, often called the Capitol or the Capitol Building, is the seat of the United States ... Capital City: Washington, D.C.. Explore the nation's capital as viewed through the collections from the U.S. Capitol to museums and monuments. The United States Capitol in Washington, D.C., is a symbol of the American people and our government and the meeting place of the nation's legislature, ... On July 16, 1790, Washington, DC was established as the nation's capital by the Constitution of the United States. You can read the actual line at the National ... Capital of the United States of America Washington, DC, the U.S. capital, is a compact city on 

In [74]:
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_user = os.getenv("PUSHOVER_USER")
pushover_url ="https://api.pushover.net/1/messages.json"

def push(text: str):
    """Send a push notification to the user """ 
    requests.post(pushover_url, data={"token": pushover_token, "user": pushover_user, "message":text})

In [75]:
tool_push=Tool(
    name="send_push_notifications",
    func=push,
    description= "Useful for when you need to send a push notification"
)

tool_push.invoke("Hello, me")

In [89]:
tools = [tool_search, tool_push]

In [90]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

In [91]:
graph_builder = StateGraph(State)

In [92]:
llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools)

In [93]:
def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", ToolNode(tools=tools))

In [94]:



graph_builder.add_conditional_edges( "chatbot", tools_condition, "tools")

graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

In [96]:
graph = graph_builder.compile()
display(Image(graph.get_graph().draw_mermaid_png()))



ValueError: Failed to reach https://mermaid.ink/ API while trying to render your graph. Status code: 204.

To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`

In [98]:
def chat(user_input: str, history):
    initial_stage =State(messages=[{"role":"user", "content":user_input}])
    result = graph.invoke(initial_stage)
    return result["messages"][-1].content

In [99]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


In [100]:
# ADDING MEMORY
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()

In [102]:
# Steps 1 and 2
graph_builder = StateGraph(State)


# Step 3
llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    print(state)
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", ToolNode(tools=tools))

# Step 4
graph_builder.add_conditional_edges( "chatbot", tools_condition, "tools")
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

# Step 5
graph = graph_builder.compile(checkpointer=memory)
# display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
config = {"configurable": {"thread_id": "1"}}

def chat(user_input: str, history):
    result = graph.invoke({"messages": [{"role": "user", "content": user_input}]}, config=config)
    return result["messages"][-1].content


gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.


{'messages': [HumanMessage(content='hi', additional_kwargs={}, response_metadata={}, id='6bbef120-e241-4652-9be9-1a93b0339939')]}
{'messages': [HumanMessage(content='hi', additional_kwargs={}, response_metadata={}, id='6bbef120-e241-4652-9be9-1a93b0339939'), AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 83, 'total_tokens': 93, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CNMfREepLZ3IfrmNjEoyJnGGidNtJ', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--6f36d340-cba3-46b4-ba85-9d312ce7e54b-0', usage_metadata={'input_tokens': 83, 'output_tokens': 10, 'total_tokens': 93, 'input_token_det

In [104]:
graph.get_state(config)

StateSnapshot(values={'messages': [HumanMessage(content='hi', additional_kwargs={}, response_metadata={}, id='6bbef120-e241-4652-9be9-1a93b0339939'), AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 83, 'total_tokens': 93, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CNMfREepLZ3IfrmNjEoyJnGGidNtJ', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--6f36d340-cba3-46b4-ba85-9d312ce7e54b-0', usage_metadata={'input_tokens': 83, 'output_tokens': 10, 'total_tokens': 93, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}), HumanMessage(co

In [105]:
# Most recent first

list(graph.get_state_history(config))

[StateSnapshot(values={'messages': [HumanMessage(content='hi', additional_kwargs={}, response_metadata={}, id='6bbef120-e241-4652-9be9-1a93b0339939'), AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 83, 'total_tokens': 93, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CNMfREepLZ3IfrmNjEoyJnGGidNtJ', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--6f36d340-cba3-46b4-ba85-9d312ce7e54b-0', usage_metadata={'input_tokens': 83, 'output_tokens': 10, 'total_tokens': 93, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}), HumanMessage(c

In [106]:
#STORING IN SQL
import sqlite3
from langgraph.checkpoint.sqlite import SqliteSaver

db_path = "memory.db"
conn = sqlite3.connect(db_path, check_same_thread=False)
sql_memory = SqliteSaver(conn)

In [107]:
# Steps 1 and 2
graph_builder = StateGraph(State)


# Step 3
llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    print(state)
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", ToolNode(tools=tools))

# Step 4
graph_builder.add_conditional_edges( "chatbot", tools_condition, "tools")
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

# Step 5
graph = graph_builder.compile(checkpointer=sql_memory)
# display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
config = {"configurable": {"thread_id": "3"}}

def chat(user_input: str, history):
    result = graph.invoke({"messages": [{"role": "user", "content": user_input}]}, config=config)
    return result["messages"][-1].content


gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.


{'messages': [HumanMessage(content='HELLO papi', additional_kwargs={}, response_metadata={}, id='9ba2a394-cbf8-4978-bd14-04c1dd506640')]}
{'messages': [HumanMessage(content='HELLO papi', additional_kwargs={}, response_metadata={}, id='9ba2a394-cbf8-4978-bd14-04c1dd506640'), AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 85, 'total_tokens': 95, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CNMi87xdfwX6XwKPhCRzoq38IkoJ9', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--35d6fb6b-d7ea-402e-a552-81ff5f4cd406-0', usage_metadata={'input_tokens': 85, 'output_tokens': 10, 'total_tokens': 95, 